In [1]:
#This is a very clunky way to "parallelize" lemmatization--
#by running this script with different settings of 'i' on separate spun-up instances of one-node, saving them, and later combining them

import pandas as pd
import pickle
import os
import nltk
import numpy as np


import TextCleaning_updated
import LDAvariables
import stanza
import gensim

# load saved df.  df['working_abstract'] contains clean text.
df = pd.read_pickle("./clean_dataset.pkl")
df.reset_index(inplace = True)
df.rename(columns={'index':'original index'}, inplace=True)

batch_size=len(df)/20
i=2 #I ran this in twenty instances, where the value of i changes and then all datasets are combined

wa='working_abstract'
nlp = stanza.Pipeline(lang='en',processors='tokenize,pos,lemma',tokenize_batch_size=500,lemma_batch_size=500)

    
def lemmatize_stanford(doc,pretokened=False,keep_numbers=True):
    """lemmatizes and tokenizes an abstract, keeping only lemmas that are noun, verb, adj, or adverb,
    with option to include numbers, and non-lemmaed pronouns, 'X', and 'intj' if pretokened, dont use this function, as it hasnt been adapted for it"""
    assert not pretokened
    new_tokens=[]
    if doc==' ':
        return np.nan
    else: 
        processed=nlp(doc)
        for sent in processed.sentences:
            for word in sent.words:
                if word.pos in ['NOUN','VERB','ADJ','ADV']:
                    new_tokens.append(word.lemma)
                elif word.pos=='NUM' and keep_numbers:
                    new_tokens.append(word.lemma)
                elif word.pos in ['PROPN','X','INTJ']: 
                    new_tokens.append(word.text)
        return new_tokens

#to compare these functions, try test cases 463, 40, and 2247 (iloc)

#To generate a batch, uncomment next two lines
tokened_lemma_docs=df[wa].iloc[int(i*batch_size):int((i+1)*batch_size)].apply(lambda x: lemmatize_stanford(x,keep_numbers=False))
#tokened_lemma_docs.to_pickle("src/prnd/publ/icrd/data/prd/RND Topic Modelling/lemma_docs/lemma_docs_"+str(i)+".pkl")

2020-06-30 17:11:59 INFO: Loading these models for language: en (English):
| Processor | Package |
-----------------------
| tokenize  | ewt     |
| pos       | ewt     |
| lemma     | ewt     |

2020-06-30 17:11:59 INFO: Use device: cpu
2020-06-30 17:11:59 INFO: Loading: tokenize
2020-06-30 17:11:59 INFO: Loading: pos
2020-06-30 17:12:00 INFO: Loading: lemma
2020-06-30 17:12:00 INFO: Done loading processors!


KeyboardInterrupt: 